# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [4]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [112]:
!pip install langchain-huggingface langchain langchain-community langchain-chroma Chroma langchain-groq pyarabic

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [113]:
groq_api_key='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'

In [130]:
import pandas as pd
df = pd.read_csv('/content/Dataset.csv')

In [131]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [132]:
from pyarabic.araby import strip_tashkeel, strip_harakat, strip_lastharaka, strip_diacritics

def clean_text(text):
  text = strip_tashkeel(text)
  text = strip_harakat(text)
  text = strip_lastharaka(text)
  text = strip_diacritics(text)
  return text

In [133]:
df['الغرامة'] = df['الغرامة'].apply(clean_text)
df['المخالفة'] = df['المخالفة'].apply(clean_text)

In [134]:
markdown_files = []

for i in range(0, len(df)):
  title = df['الغرامة'].iloc[i]
  content = df['المخالفة'].iloc[i]

  markdown_content = f"*{title}* {content}"

  markdown_files.append(markdown_content)



In [135]:
markdown_files

['*الغرامة المالية 100 - 150 ريال* قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.',
 '*الغرامة المالية 100 - 150 ريال* ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.',
 '*الغرامة المالية 100 - 150 ريال* عدم وجود تأمين ساري للمركبة.',
 '*الغرامة المالية 100 - 150 ريال* عبور المشاة للطرق من غير الأماكن المخصصة لهم.',
 '*الغرامة المالية 100 - 150 ريال* عدم تقيد المشاة بالإشارات الخاصة بهم.',
 '*الغرامة المالية 100 - 150 ريال* وقوف المركبة في أماكن غير مخصصة للوقوف.',
 '*الغرامة المالية 100 - 150 ريال* عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.',
 '*الغرامة المالية 150 - 300 ريال* عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.',
 '*الغرامة المالية 150 - 300 ريال* الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين مترا.',
 '*الغرامة المالية 150 - 300 ريال* قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.',
 '*الغرامة المالية 150 - 300 ريال* 

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [136]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter =RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
documents=text_splitter.create_documents(markdown_files)

In [137]:
documents[0]

Document(metadata={}, page_content='*الغرامة المالية 100 - 150 ريال* قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.')

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [138]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
embeddings = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")
db = Chroma.from_documents(documents, embeddings, persist_directory='./Chroma_db')

In [139]:
data = Chroma(
    persist_directory='./Chroma_db',
    embedding_function=embeddings,
)

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [140]:
from langchain.prompts import PromptTemplate
temp = """  مرحبا ذكاء اصطناعي اعط اجابات  للمخالفة  بناءا على الغرامة
------
المخالفة:{violation}
------
الغرامة:{fine}
"""

prompt = PromptTemplate(
    template=temp,
    input_variables=["violation", "fine"],
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [141]:
from langchain_groq import ChatGroq
llm=ChatGroq(temperature=0, api_key=groq_api_key, model_name="llama-3.1-70b-versatile", verbose=1)

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [142]:
from langchain import LLMChain
model = LLMChain(prompt=prompt, llm=llm, verbose=1)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [143]:
def query_rag(qeury: str):
  similarity_search = data.similarity_search_with_relevance_scores(qeury, k=6)
  print(similarity_search[1])
  context_text = ""
  for result in similarity_search:
    context_text += result[0].page_content + '\n \n'
  rag_res = model.invoke({'fine':context_text, 'violation':qeury})
  return rag_res

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [144]:
print(query_rag('ماهي الغرامة على القيادة بدون رخصة؟'))

<ipython-input-143-cea9635cc09f>:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='*الغرامة المالية 1000 - 2000 ريال* قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.'), -197.62065835180368), (Document(metadata={}, page_content='*الغرامة المالية 1000 - 2000 ريال* قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.'), -197.62070027997524), (Document(metadata={}, page_content='*الغرامة المالية 150 - 300 ريال* عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.'), -206.01230763608962), (Document(metadata={}, page_content='*الغرامة المالية 150 - 300 ريال* عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.'), -206.01230763608962), (Document(metadata={}, page_content='*الغرامة المالية 500 - 900 ريال* قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.'), -226.81440599086375), (Document(metadata={}, page_content='*الغرامة المالية 500 - 900 ريال* قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.'), -226.81441816156354)

(Document(metadata={}, page_content='*الغرامة المالية 1000 - 2000 ريال* قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.'), -197.62070027997524)
Prompt after formatting:
  اعط اجابات  للمخالفة  بناءا على الغرامة
------
المخالفة:ماهي الغرامة على القيادة بدون رخصة؟ 
------
الغرامة:*الغرامة المالية 1000 - 2000 ريال* قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.
 
*الغرامة المالية 1000 - 2000 ريال* قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.
 
*الغرامة المالية 150 - 300 ريال* عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.
 
*الغرامة المالية 150 - 300 ريال* عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.
 
*الغرامة المالية 500 - 900 ريال* قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.
 
*الغرامة المالية 500 - 900 ريال* قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.
 



> Finished chain.
{'fine': '*الغرامة المالية 1000 - 2000 ريال* قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.\n \n*الغرامة المالية 10

In [145]:
print(query_rag('عدم الالتزام بحدود المسارات المحددة على الطريق؟'))

<ipython-input-143-cea9635cc09f>:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='*الغرامة المالية 300 - 500 ريال* عدم التقيد بتنظيمات السير عند تقاطعات الطرق.'), -205.64294845255796), (Document(metadata={}, page_content='*الغرامة المالية 300 - 500 ريال* عدم التقيد بتنظيمات السير عند تقاطعات الطرق.'), -205.64297986021046), (Document(metadata={}, page_content='*الغرامة المالية 300 - 500 ريال* عدم الالتزام بحدود المسارات المحددة على الطريق.'), -205.8393936151653), (Document(metadata={}, page_content='*الغرامة المالية 300 - 500 ريال* عدم الالتزام بحدود المسارات المحددة على الطريق.'), -205.83941755547588), (Document(metadata={}, page_content='*الغرامة المالية 500 - 900 ريال* عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على المسارات المخصصة لها.'), -227.76649473374948), (Document(metadata={}, page_content='*الغرامة المالية 500 - 900 ريال* عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو ا

(Document(metadata={}, page_content='*الغرامة المالية 300 - 500 ريال* عدم التقيد بتنظيمات السير عند تقاطعات الطرق.'), -205.64297986021046)
Prompt after formatting:
  اعط اجابات  للمخالفة  بناءا على الغرامة
------
المخالفة:عدم الالتزام بحدود المسارات المحددة على الطريق؟ 
------
الغرامة:*الغرامة المالية 300 - 500 ريال* عدم التقيد بتنظيمات السير عند تقاطعات الطرق.
 
*الغرامة المالية 300 - 500 ريال* عدم التقيد بتنظيمات السير عند تقاطعات الطرق.
 
*الغرامة المالية 300 - 500 ريال* عدم الالتزام بحدود المسارات المحددة على الطريق.
 
*الغرامة المالية 300 - 500 ريال* عدم الالتزام بحدود المسارات المحددة على الطريق.
 
*الغرامة المالية 500 - 900 ريال* عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على المسارات المخصصة لها.
 
*الغرامة المالية 500 - 900 ريال* عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على المسارات المخصصة لها.
 



> Finished chain.
{'fine': '*الغرامة المالية 300 - 500 ريال* عدم التقيد بتنظيمات 